In [1]:
import numpy as np
import pandas as pd

# make sure you have installed the Plotly package
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

# fill in your username and api_key before using Plotly
plotly.tools.set_credentials_file(username = '5005p', 
                                  api_key = 'njeG2Tu75B6pcDoUQQwL')

data = pd.read('TestCenterInfo.csv')

In [2]:
test_events = pd.read_csv('G:/5005/ACT_TestEvents.csv')
all_points = pd.read_csv('G:/5005/ACT_AllPointsFile.csv')
test_centers = pd.read_csv('G:/5005/testcenter_3.csv')
test_events['utility'] = test_events.ASSIGNED / test_events.CAPACITY

data = pd.merge(test_events, all_points, left_on = 'TestCenterID', right_on = 'TESTCENTERID', how = 'left')
data = pd.merge(data, test_centers, on = ['LATITUDE', 'LONGITUDE'], how = 'left')
data = data.rename(columns = {'TestCenterID':'test_center_id',
                       'LATITUDE':'lat',
                       'LONGITUDE':'lon',
                       'testcenterstate':'state',
                       'POPULATION':'population'})
data.YEAR = data.YEAR.apply(lambda x: str(x))
data.MONTH = data.MONTH.apply(lambda x: str(x))
data.MONTH = data.MONTH.apply(lambda x: str(0) + x if len(x) == 1 else x)
data['time'] = data.YEAR + data.MONTH
data['time'] = data.time.apply(lambda x: int(x))
data.MONTH = data.MONTH.apply(lambda x: int(x))
data.YEAR = data.YEAR.apply(lambda x: int(x))
del data['TestEventID']
del data['TESTDATE']
del data['CYCLE_DESCRIPTION']
del data['ADMIN_GROUP']
del data['TESTCENTERID']
del data['POINTID']
del data['TCTYPE']
del data['INSTITUTION']
del data['Unnamed: 0']
del data['POINTTYPE']

In [52]:
# the change curve of utilization of each states 
trace = []
for i in data.state.unique():
    state_i = data[data.state == i]
    y = []
    for j in sorted(state_i.time.unique()):
        y.append(state_i[state_i.time == j].utility.mean())
    trace_i = go.Scatter(
        x = np.array(range(len(state_i.time.unique()))),
        y = np.array(y),
        mode = "lines + markers",
        name = i)
    trace.append(trace_i)

py.iplot(trace, filename = 'change curve of utilization')


In [99]:
# distribution cure of utilization of a certain state in the first half and the second half of the year
state_i = data[data.state == 'florida']

first_half = np.array(state_i[state_i.MONTH < 6].utility)
second_half = np.array(state_i[state_i.MONTH >= 6].utility)

trace_first = go.Histogram(
    x = first_half,
    opacity = 0.75,
    name = 'first half year',
)
trace_second = go.Histogram(
    x = second_half,
    opacity = 0.55,
    name = 'second half year'
)

trace = [trace_first, trace_second]

layout = go.Layout(barmode = 'overlay')

fig = go.Figure(data = trace, layout = layout)

py.iplot(fig, filename = 'distribution curve of utilization')


In [68]:
# observe the utilization pattern in different seasons by using contour plot

x=['spring', 'summer', 'autumn' , 'winter']
y=[]
z=[]
for i in data.state.unique()[6:10]:
    y.append(i)

for i in y:
    state_i = data[data.state == i]
    state_i_spring = state_i[((state_i.MONTH>0)&(state_i.MONTH<=3))]
    z.append(state_i_spring.utility.mean())
    
for i in y:
    state_i = data[data.state == i]
    state_i_spring = state_i[((state_i.MONTH>0)&(state_i.MONTH<=6))]
    z.append(state_i_spring.utility.mean())
for i in y:
    state_i = data[data.state == i]
    state_i_spring = state_i[((state_i.MONTH>6)&(state_i.MONTH<=9))]
    z.append(state_i_spring.utility.mean())
for i in y:
    state_i = data[data.state == i]
    state_i_spring = state_i[((state_i.MONTH>9)&(state_i.MONTH<=12))]
    z.append(state_i_spring.utility.mean())
t= []
for i in range(4):
    t.append(z[i * 4:(i+1) * 4])
trace = [
    go.Contour(
        z = t,
        x = y,
        y = x
    )]

py.iplot(trace)

In [82]:
trace = []
for i in data.state.unique():
    a = np.random.randint(50,200)
    b = np.random.randint(50,200)
    c = np.random.randint(50,200)
    r = []
    t = ['FEB','APR','JUN','SEP','OCT','DEC']
    state_i = data[data.state == i]
    r.append(state_i[state_i.MONTH == 2].utility.mean())
    r.append(state_i[state_i.MONTH == 4].utility.mean())
    r.append(state_i[state_i.MONTH == 6].utility.mean())
    r.append(state_i[state_i.MONTH == 9].utility.mean())
    r.append(state_i[state_i.MONTH == 10].utility.mean())
    r.append(state_i[state_i.MONTH == 12].utility.mean())
    trace_i = go.Area(
        r = r,
        t = t,
        name = i,
        marker = dict(color = 'rgb({},{},{})'.format(a, b, c)))
    trace.append(trace_i)

layout = go.Layout(
    title = 'low utilization detection',
    font = dict(
        size = 16
    ),
    legend = dict(
        font = dict(
            size = 16
        )
    ),
    radialaxis = dict(
        ticksuffix = '%'
    ),
    orientation = 270
)
fig = go.Figure(data = trace, layout = layout)
py.iplot(fig, filename = 'low utilization detection')  

In [113]:
rawData = []
for i in data.state.unique():
    
    point = {}
    state_i = data[data.state == i]
    point['population'] = (state_i.CAPACITY.mean() - data.CAPACITY.min())/(data.CAPACITY.max()-data.CAPACITY.min())
    point['expense'] = (state_i.EXPENSE.mean() -data.EXPENSE.min())/(data.EXPENSE.max()-data.EXPENSE.min())
    point['utility'] = (state_i.ASSIGNED.mean()  - data.ASSIGNED.min())/(data.ASSIGNED.max()-data.ASSIGNED.min())
    point['label'] = i
    rawData.append(point)


def makeAxis(title, tickangle): 
    return {
      'title': title,
      'titlefont': { 'size': 20 },
      'tickangle': tickangle,
      'tickfont': { 'size': 15 },
      'tickcolor': 'rgba(0,0,0,0)',
      'ticklen': 5,
      'showline': True,
      'showgrid': True
    }

trace = [{ 
    'type': 'scatterternary',
    'mode': 'markers',
    'a': [i for i in map(lambda x: x['population'], rawData)],
    'b': [i for i in map(lambda x: x['expense'], rawData)],
    'c': [i for i in map(lambda x: x['utility'], rawData)],
    'text': [i for i in map(lambda x: x['label'], rawData)],
    'marker': {
        'symbol': 100,
        'color': '#DB7365',
        'size': 14,
        'line': { 'width': 2 }
    },
    }]

layout = {
    'ternary': {
        'sum': 100,
        'aaxis': makeAxis('population', 0),
        'baxis': makeAxis('<br>expense', 45),
        'caxis': makeAxis('<br>utility', -45)
    },
    'annotations': [{
      'showarrow': False,
      'text': 'Simple Ternary Plot with Markers',
        'x': 0.5,
        'y': 1.3,
        'font': { 'size': 15 }
    }]
}

fig = {'data': trace, 'layout': layout}
py.iplot(fig, validate=False)

In [133]:
trace = []
c = ['peru','darkviolet','deepskyblue','orangered','green']
j=0
for i in data.state.unique()[0:10]:
    state_i = data[data.state == i]
    a = np.random.randint(50,235)
    b = np.random.randint(50,240)
    c = np.random.randint(50,240)
    trace_i = go.Scatter(
        r = np.array(state_i.ASSIGNED),
        t = np.array(state_i.CAPACITY),
        mode = 'markers',
        name = i,
        marker = dict(
            color='rgb({},{},{})'.format(a,b,c),
            size=110,
            line=dict(
            color='white'

            ),
         opacity=0.7
    )
    )
    trace.append(trace_i)

layout = go.Layout(
    title='Hobbs-Pearson Trials',
    font=dict(
        size=15
    ),
    plot_bgcolor='rgb(223, 223, 223)',
    angularaxis=dict(
        tickcolor='rgb(253,253,253)'
    )
)
fig = go.Figure(data=trace, layout=layout)
py.iplot(fig)

In [36]:
all_states = pd.read_csv('G:/5005/allstate.csv',usecols=[3,4,9,10],header=0,names=['LATITUDE','LONGITUDE','population','state'])

In [40]:
len(all_states)

39118

In [48]:
data2.head()
data2=data2.rename(columns={'state':'name',"population":'pop',"LONGITUDE":'lon',"LATITUDE":"lat"})
data2.head()


,lat,lon,pop,name
4,33.528867,-117.701175,4266.0,california
8,33.467371,-117.613509,2014.0,california
9,33.501860,-117.608646,2713.0,california
10,33.547238,-117.623761,1080.0,california
11,33.387723,-117.444758,2921.0,california


In [61]:
import plotly.plotly as py
import pandas as pd

df=data2

df['text'] = df['name'] + '<br>Population ' + (df['pop']).astype(str)+' million'
limits = [(0,500),(500,2000),(2000,4000),(4000,9000),(9000,13000)]
#colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey","rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
colors = []
for i in range(len(limits)):
    colors.append("rgb({},{},{})".format(np.random.randint(100,220),np.random.randint(100,220),np.random.randint(100,220)))
cities = []
scale = 100

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    city = dict(
        type = 'scattergeo',
        locationmode = 'USA-states',
        lon = df_sub['lon'],
        lat = df_sub['lat'],
        text = df_sub['text'],
        marker = dict(
            size = df_sub['pop']/scale,
            color = colors[i],
            line = dict(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    cities.append(city)

layout = dict(
        title = '2014 US city populations<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
py.iplot( fig, validate=False, filename='d3-bubble-map-populations' )

In [52]:
data2['pop'].max()

12822.0